### Import 套件

In [7]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### 設定路徑

In [8]:
ROOT_DIR = os.getcwd() #main directory path
IMG_DIR = os.path.join(ROOT_DIR,'data','images') #images path
MODEL_DIR = os.path.join(ROOT_DIR, 'model') #model path
CASCADE_PATH = os.path.join(MODEL_DIR, 'cv2', 'haarcascade_frontalface_alt2.xml') #cv2 path
KERAS_PATH = os.path.join(MODEL_DIR, 'keras', 'facenet_keras.h5')
image_size = 160
model = load_model(KERAS_PATH)

### 影像白化處理，去除影像中多餘重複資訊，亦可突顯特徵部份 ,定義 L2

In [9]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

### 圖片處理

In [10]:
def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(CASCADE_PATH)
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

### 計算embedding值

In [11]:
def calc_embs(filepaths, margin=10, batch_size=1):   ### !!!! 這裡要修改
    aligned_images = prewhiten(load_and_align_images(filepaths, margin))
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs

#### 測試

In [12]:
calc_embs(['/home/user/Downloads/keras-facenet/data/images/TsaiYingWen/Tsai_Ying_Wen0000.jepg'])  ###!!!

array([[ 8.24472979e-02, -2.35767499e-01,  1.64598139e-04,
        -2.18825787e-01, -1.36992753e-01, -7.49433264e-02,
         3.24381776e-02, -2.20765807e-02, -4.95827533e-02,
        -1.36791933e-02,  3.13692726e-02,  1.75563004e-02,
        -5.17494678e-02,  1.42397294e-02,  3.84938791e-02,
         6.39858097e-02,  4.34267484e-02, -1.11139715e-01,
         6.87315315e-03, -3.62205924e-03,  2.77963858e-02,
        -1.36720806e-01, -1.37045994e-01, -1.57446600e-02,
         9.30140615e-02,  3.13901454e-02,  3.35380137e-02,
         1.24795422e-01,  5.73032014e-02, -4.74012131e-03,
         1.74248852e-02,  2.62233526e-01,  4.44929227e-02,
        -1.02115802e-01, -4.44795489e-02,  1.80017296e-02,
        -2.01353267e-01,  1.26397282e-01,  2.04484150e-01,
         7.22195655e-02, -1.12205874e-02,  1.33577451e-01,
         1.07557118e-01, -1.74560100e-01, -3.88017409e-02,
        -5.74937649e-02, -1.24023266e-01,  6.50344416e-02,
        -2.05163389e-01,  6.01701811e-02,  6.88953996e-0

### 定義計算歐式距離

In [13]:
def calc_dist1(imgemb0,imgemb1):
    emb0 = [imgemb0]
    emb1 = [imgemb1]
    return distance.euclidean(calc_embs(emb0), calc_embs(emb1))

### 定義計算各目錄圖片並排序

In [14]:
def img_dist(image_dirpath): #output a list of img distances in img_path
    img_path = os.listdir(image_dirpath)    #讀出image_dirpath下的資料夾
    for i in range(len(img_path)):
        name_paths = os.path.join(image_dirpath,str(img_path[i]))  #用迴圈依序讀出各資料夾並拼接出絕對路徑
        sort_list = []                                             #/home/user/Downloads/keras-facenet/data/images/ + BillGates
        name_imgpaths = [os.path.join(name_paths, f) for f in os.listdir(name_paths)]#因為listdir只能讀路徑無法讀列表 所以必須把name_paths組合成絕對路徑
                                                                    #讀取目錄下的圖片再為他們拼接成絕對路徑#'/home/user/Downloads/keras-facenet/data/images/BillGates/ + Bill_Gates_0002 (copy).jpg'
        embs = calc_embs(name_imgpaths)
        n =len(name_imgpaths)
        print(n)
        for j in range(n-1):
            for k in range(n-j-1):
                #print(j,j+k+1)
                result = calc_dist1(name_imgpaths[j], name_imgpaths[j+k+1])
                sort_list.append(result)
        print(sorted(sort_list, key=float))
    return sorted(sort_list, key=float),embs

In [15]:
(ll, emb) = img_dist('/home/user/Downloads/keras-facenet/data/images')

4
[0.0, 0.5930255055427551, 0.5930255055427551, 0.618617832660675, 0.7355362176895142, 0.7355362176895142]
2
[0.6874032020568848]
3
[0.4924164414405823, 0.5091053247451782, 0.569210410118103]
6
[0.426786869764328, 0.42865628004074097, 0.4302254319190979, 0.44833168387413025, 0.46876955032348633, 0.5303853154182434, 0.5716536045074463, 0.6159211993217468, 0.6398237347602844, 0.6648781895637512, 0.6650308966636658, 0.6675175428390503, 0.7151914238929749, 0.7174002528190613, 0.7460352182388306]
3
[0.5079659819602966, 0.6593101620674133, 0.7331599593162537]
7
[0.49096742272377014, 0.5101138353347778, 0.7157517671585083, 0.7355362176895142, 0.7434427738189697, 1.1632869243621826, 1.1636340618133545, 1.2159498929977417, 1.2465523481369019, 1.264832854270935, 1.3112479448318481, 1.3117632865905762, 1.3312784433364868, 1.3754124641418457, 1.3787062168121338, 1.4008688926696777, 1.4101545810699463, 1.4120597839355469, 1.4303568601608276, 1.4319829940795898, 1.482345700263977]


In [16]:
ll

[0.49096742272377014,
 0.5101138353347778,
 0.7157517671585083,
 0.7355362176895142,
 0.7434427738189697,
 1.1632869243621826,
 1.1636340618133545,
 1.2159498929977417,
 1.2465523481369019,
 1.264832854270935,
 1.3112479448318481,
 1.3117632865905762,
 1.3312784433364868,
 1.3754124641418457,
 1.3787062168121338,
 1.4008688926696777,
 1.4101545810699463,
 1.4120597839355469,
 1.4303568601608276,
 1.4319829940795898,
 1.482345700263977]

In [17]:
emb

array([[-9.48515311e-02, -7.10194334e-02, -5.68938442e-02,
        -2.95100342e-02,  7.29728937e-02,  1.03596821e-01,
        -2.89175976e-02,  8.89702216e-02,  1.36376128e-01,
        -7.75502771e-02,  1.14509920e-02, -1.57378510e-01,
        -1.05415598e-01,  2.64471862e-02,  6.57941252e-02,
         2.66889054e-02,  1.32591531e-01, -4.12595272e-02,
        -3.67686078e-02,  7.79124424e-02,  4.92879003e-02,
        -4.34372053e-02,  1.25753451e-02,  1.08075291e-01,
        -1.31131709e-01,  7.89197460e-02,  1.24709465e-01,
         1.00495825e-02, -1.44922897e-01,  3.02652158e-02,
        -2.01089773e-02,  9.49468762e-02, -8.54681879e-02,
         5.39316311e-02,  3.35794054e-02,  1.06426306e-01,
        -3.49620208e-02, -1.30541548e-01,  1.08218089e-01,
        -2.06421781e-02, -1.11312054e-01, -2.06577569e-01,
        -6.43938780e-02, -2.44294375e-01,  3.90483625e-02,
        -7.23963827e-02,  1.42472580e-01, -4.54731025e-02,
        -9.04010683e-02,  1.15637176e-01, -6.80933669e-0